In [ ]:
# Data Preprocessing - check if Standard Sxcaler will work well
# Alternatively, divide by 200?
# source: https://medium.com/analytics-vidhya/introduction-to-neural-networks-for-finance-6abd5675e497

In [1]:
# break up the data into training sets
def gen_train(self, seq_len):
    """
    Generates training data
    :param seq_len: length of window
    :return: X_train and Y_train
    """
    for i in range((len(self.stock_train)//seq_len)*seq_len - seq_len - 1):
        x = np.array(self.stock_train.iloc[i: i + seq_len, 1])
        y = np.array([self.stock_train.iloc[i + seq_len + 1, 1]], np.float64)
        self.input_train.append(x)
        self.output_train.append(y)
    self.X_train = np.array(self.input_train)
    self.Y_train = np.array(self.output_train)

In [3]:
import tensorflow as tf

In [4]:
# Multilayer Perceptron (MLP) 
# source: https://medium.com/engineer-quant/multilayer-perceptron-4453615c4337
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(100, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(100, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.relu))
model.compile(optimizer="adam", loss="mean_squared_error")

# Adam optimizer is gaining popularity in the machine learning community because it is 
#a more efficient algorithm to optimize compared to traditional stochastic gradient descent. 
#The advantages are best understood by looking at  the advantages of two other extensions of
#stochastic gradient descent 
#(https://medium.com/analytics-vidhya/introduction-to-neural-networks-for-finance-6abd5675e497)

model.fit(X_train, Y_train, epochs=100)
model.evaluate(X_test, Y_test)






# Long Short Term Model (LSTM)
# source: https://www.altumintelligence.com/articles/a/Time-Series-Prediction-Using-LSTM-Deep-Neural-Networks

W0904 09:38:51.195810 4690433472 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


NameError: name 'X_train' is not defined

In [5]:
# Long Short Term Model (LSTM)
# source: https://www.altumintelligence.com/articles/a/Time-Series-Prediction-Using-LSTM-Deep-Neural-Networks

model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(20, input_shape=(10, 1), return_sequences=True))
model.add(tf.keras.layers.LSTM(20))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.relu))
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, Y_train, epochs=50)
model.evaluate(X_test, Y_test)

NameError: name 'X_train' is not defined

In [ ]:
# Note on Backtesting - check how to do it?

#Example below. For full blown backtest systems, you will need to consider factors such as 
#survivorship bias, look ahead bias, market regime change and transaction costs.

#Now that we have fitted our models using our training data and evaluated it using our test data, 
#we can take the assessment a step further by backtesting the model on new data. Backtesting is 
#essentially running your strategy (or i our case, the prediction algorithm) over data from a period 
#of time to see the profit and loss, or accuracy of the algorithm. This is done simply by

def back_test(strategy, seq_len, ticker, start_date, end_date, dim):
    """
    A simple back test for a given date period
    :param strategy: the chosen strategy. Note to have already formed the model, and fitted with training data.
    :param seq_len: length of the days used for prediction
    :param ticker: company ticker
    :param start_date: starting date
    :type start_date: "YYYY-mm-dd"
    :param end_date: ending date
    :type end_date: "YYYY-mm-dd"
    :param dim: dimension required for strategy: 3dim for LSTM and 2dim for MLP
    :type dim: tuple
    :return: Percentage errors array that gives the errors for every test in the given date range
    """
    data = pdr.get_data_yahoo(ticker, start_date, end_date)
    stock_data = data["Adj Close"]
    errors = []
    for i in range((len(stock_data)//10)*10 - seq_len - 1):
        x = np.array(stock_data.iloc[i: i + seq_len, 1]).reshape(dim) / 200
        y = np.array(stock_data.iloc[i + seq_len + 1, 1]) / 200
        predict = strategy.predict(x)
        while predict == 0:
            predict = strategy.predict(x)
        error = (predict - y) / 100
        errors.append(error)
        total_error = np.array(errors)
    print(f"Average error = {total_error.mean()}")
